In [45]:
import numpy as np
import os
frames = []

for map_frames in os.listdir('frames/wi'):
    print(map_frames)
    uncompressed = np.load(os.path.join('frames', 'wi', map_frames))
    frames += list(uncompressed['kwds'].reshape(1)[0].values())

a31002f494bf.npz
e65ddf30a0d6.npz
e65d06674ec5.npz
352aad0346f9.npz
5f413f9e2b13.npz
afca69875bae.npz
6dffbd562ede.npz
33923212ec06.npz
713e148d66cb.npz
779675b51a51.npz
1be0b3cfc81f.npz
6af6e45693e2.npz
0182ae74c8b0.npz
3084c24c266f.npz
4f4c2950f070.npz
8bccc215ef38.npz
ce67c9ccb408.npz
d6038b5dd437.npz
71768cc6cf35.npz
c2e7eaa8358a.npz
c78adc25f9f4.npz
c08bedfc58e4.npz
21d0def62efc.npz
052db0c448bb.npz
17d380d4b56e.npz
7699100b8751.npz
189f4d10e46f.npz
c35a3ef3e105.npz
3f557c27b56c.npz
23306e87e8f9.npz
02d1cdc3d7fb.npz
6fbf95329302.npz
d2b981403d0d.npz
29266e49ea13.npz
93c252db988c.npz
d7df6e5c2cc3.npz
556df7fadf8b.npz
148e1a7ed88c.npz
3b6164b9dbb8.npz
a0a88a99d929.npz
c672faddc6b7.npz
c0d587c0cda5.npz
21e543261cd9.npz
fb68418f9e93.npz
b148699b7b9e.npz
20a0c655dccc.npz
a0c4ad7b32a8.npz
c04a0a7a9e64.npz
8891ae21560a.npz
86be26636557.npz
07fe685ba184.npz
c9fb41e933a9.npz
6c07245f52e9.npz
67d62441111a.npz
0ad4abcf63f7.npz
b7f91748ac4f.npz
ec9465dc181b.npz
e9752ce3df45.npz
59d8c981289f.n

In [54]:
# based on https://blog.keras.io/building-autoencoders-in-keras.html
# https://nbviewer.jupyter.org/github/autonomio/talos/blob/master/examples/Hyperparameter%20Optimization%20with%20Keras%20for%20the%20Iris%20Prediction.ipynb
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D
from keras.models import Model
from keras import backend as K
from talos.model.early_stopper import early_stopper
import talos as ta

params = {
    'conv1_filters':  (4, 32, 8),
    'conv1_ksize':    [3, 5, 7],
    'max_pool_1':     [2, 3, 4],
    'conv2_filters':  (4, 16, 4),
    'conv2_ksize':    [3, 5, 7],
    'max_pool_2':     [2, 3, 4],
    'conv3_filters':  (4, 16, 4),
    'conv3_ksize':    [3, 5, 7],
    'max_pool_3':     [2, 3, 4],
    'conv0_ksize':    [3, 5, 7], # decoder only
    'optimizer':      ['adadelta'],
    'batch_size':     [2048],
    'epochs':         [300]
}

def ae_model(x_train, y_train, x_val, y_val, params):
    input_img = Input(shape=x_train[0].shape)
    x = Conv2D(params['conv1_filters'], (params['conv1_ksize'], params['conv1_ksize']), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((params['max_pool_1'], params['max_pool_1']), padding='same')(x)
    x = Conv2D(params['conv2_filters'], (params['conv2_ksize'], params['conv2_ksize']), activation='relu', padding='same')(x)
    x = MaxPooling2D((params['max_pool_2'], params['max_pool_2']), padding='same')(x)
    x = Conv2D(params['conv3_filters'], (params['conv3_ksize'], params['conv3_ksize']), activation='relu', padding='same')(x)
    encoded = MaxPooling2D((params['max_pool_3'], params['max_pool_3']), padding='same')(x)

    x = Conv2D(params['conv3_filters'], (params['conv3_ksize'], params['conv3_ksize']), activation='relu', padding='same')(encoded)
    x = UpSampling2D((params['max_pool_3'], params['max_pool_3']))(x)
    x = Conv2D(params['conv2_filters'], (params['conv2_ksize'], params['conv2_ksize']), activation='relu', padding='same')(x)
    x = UpSampling2D((params['max_pool_2'], params['max_pool_2']))(x)
    x = Conv2D(params['conv1_filters'], (params['conv1_ksize'], params['conv1_ksize']), activation='relu')(x)
    x = UpSampling2D((params['max_pool_1'], params['max_pool_1']))(x)
    decoded = Conv2D(1, (params['conv0_ksize'], params['conv0_ksize']), activation='sigmoid', padding='same')(x)
    
    ae = Model(input_img, decoded)
    ae.compile(optimizer=params['optimizer'], loss='mean_absolute_error')
    out = ae.fit(x_train, y_train, batch_size=params['batch_size'], epochs=params['epochs'], verbose=0,
                 validation_data=[x_val, y_val], callbacks=early_stopper(params['epochs'], mode='strict'))

frames = np.array(frames)
h = ta.Scan(frames, frames, params=params, model=ae_model, dataset_name='wi_frames', experiment_no='1', grid_downsample=0.01)

KeyboardInterrupt: 

In [48]:
frames[0].shape

(265, 248)